In [12]:
import pyomo.environ as pyo
from idaes.core import FlowsheetBlock, declare_process_block_class, UnitModelBlockData
from idaes.core.solvers import get_solver
from idaes_ui.fv import visualize
from pyomo.environ import Var, Constraint, units as pyunits



In [13]:
# Declare the unit model block class for AmmoniaCracker
@declare_process_block_class("AmmoniaCracker")
class AmmoniaCrackerData(UnitModelBlockData):
    def build(self):
        super().build()  # Call the parent build method to initialize the unit model
        
        # Define variables for the AmmoniaCracker unit
        self.flow_ammonia = Var(initialize=1.0, units=pyunits.mol/pyunits.s, doc="Ammonia feed flow rate")
        self.temperature = Var(initialize=800, units=pyunits.K, doc="Reactor temperature")
        self.pressure = Var(initialize=101325, units=pyunits.Pa, doc="Reactor pressure")
        self.conversion = Var(initialize=0.9, bounds=(0, 1), doc="Fraction of ammonia converted")
        self.flow_hydrogen = Var(initialize=1.5, units=pyunits.mol/pyunits.s, doc="Hydrogen produced")
        self.flow_nitrogen = Var(initialize=0.5, units=pyunits.mol/pyunits.s, doc="Nitrogen produced")

        # Define constraints for the AmmoniaCracker
        @self.Constraint(doc="Ammonia conversion constraint")
        def reaction_balance(b):
            return b.flow_hydrogen == 1.5 * b.flow_ammonia * b.conversion

        @self.Constraint(doc="Nitrogen balance constraint")
        def nitrogen_balance(b):
            return b.flow_nitrogen == 0.5 * b.flow_ammonia * b.conversion


In [14]:
# Create the ConcreteModel
m = pyo.ConcreteModel()

# Initialize the flowsheet
m.fs = FlowsheetBlock(dynamic=False)

# Add the unit model (AmmoniaCracker) to the flowsheet
m.fs.cracker = AmmoniaCracker()


In [19]:

# Define dummy flow variables to simulate input/output (for visualization)
m.fs.inlet_flow = Var(initialize=1.0, units=pyunits.mol/pyunits.s, doc="Dummy inlet flow")
m.fs.outlet_flow = Var(initialize=1.0, units=pyunits.mol/pyunits.s, doc="Dummy outlet flow")

In [20]:
# Fix the input values (these are known or specified values)
m.fs.cracker.flow_ammonia.fix(1.0)   # Ammonia flow rate (mol/s)
m.fs.cracker.temperature.fix(800)    # Reactor temperature (K)
m.fs.cracker.pressure.fix(101325)    # Reactor pressure (Pa)
m.fs.cracker.conversion.fix(0.9)     # Ammonia conversion (90%)


In [22]:
m.fs.cracker.inlet_flow = m.fs.inlet_flow
m.fs.cracker.outlet_flow = m.fs.outlet_flow

RuntimeError: Re-assigning the component 'inlet_flow' from block 'fs' to
block 'fs.cracker' as 'inlet_flow'.

This behavior is not supported by Pyomo; components must have a
single owning block (or model), and a component may not appear
multiple times in a block.  If you want to re-name or move this
component, use the block del_component() and add_component() methods.


In [17]:
# Use the solver from IDAES
solver = get_solver()

# Solve the model and print the results
solver.solve(m, tee=True)


Ipopt 3.13.2: nlp_scaling_method=gradient-based
tol=1e-06
max_iter=200


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes fo

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 2, 'Number of variables': 2, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.13.2\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 0.08252096176147461}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [18]:
# Visualize the AmmoniaCracker unit model
visualize(m.fs.cracker)


2025-03-26 11:12:57 [INFO] idaes.idaes_ui.fv.fsvis: Using HTTP server on localhost, port 49826
2025-03-26 11:12:57 [INFO] idaes.idaes_ui.fv.fsvis: Loading saved flowsheet from 'flowsheet.json'
2025-03-26 11:12:57 [INFO] idaes.idaes_ui.fv.fsvis: Saving flowsheet to default file 'flowsheet.json' in current directory (c:\Users\Sara\Desktop\IDAES)
2025-03-26 11:12:57 [WARNING] idaes.idaes_ui.fv.flowsheet: Flowsheet has no Arcs or unit Blocks
2025-03-26 11:12:57 [INFO] idaes.idaes_ui.fv.fsvis: Flowsheet visualization at: http://localhost:49826/app?id=flowsheet


VisualizeResult(store=<idaes_ui.fv.persist.FileDataStore object at 0x00000282FB5985B0>, port=49826, server=<idaes_ui.fv.model_server.FlowsheetServer object at 0x00000282FA0023B0>)